<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=806a674cf80bba3a78e71c3b07064d08d23fbb3dcab94aa694c42465eea6c8b4
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-03 09:15:43
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: 500.00 (0.0%)
Current PnL: -23.44 L (-15.36%)
CY Booked + Current PnL: -9.55 L (-6.26%)
-------------------
Total profit:  1.52 L
Total loss:  -24.96 L
-------------------
Total Booked + Current PnL: 17.43 L (13.87%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.76%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.85 L (62.42%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.02%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.25,-8.91,9.80,0.02,15582.0,-15554.0,159005.0,142.75,66.0,M-SC,14.25,234.0,-1.00,1.12,38.31,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.61,-16.05,19.22,0.09,16261.0,-16171.0,84606.0,96.37,42.0,M-SC,2.97,253.0,-0.99,0.59,11.15,OX40N,NTT,DURABLES
51,MASFIN,398.61,0.22,-2.34,24.98,22.05,23902.0,-2295.0,95685.0,-15.99,59.0,H-SC,6.72,164.0,-0.10,0.67,39.09,XR,ATH,FINANCE
43,ITC,452.00,-0.04,-2.07,12.77,10.44,25028.0,-4147.0,195991.0,-41.56,40.0,X-LC,1.51,5.0,-0.17,1.38,3.38,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.22,5.76,13.65,20.19,25164.0,10034.0,184354.0,-17.20,47.0,X-MC,6.75,68.0,0.40,1.30,21.74,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
13,BSOFT,831.70,5.06,-26.58,105.38,50.79,108817.0,-37391.0,103262.0,-5.68,65.0,H-SC,6.39,151.0,-0.34,0.73,20.23,XR,ATH,IT
48,KPIGREEN,730.66,1.76,-12.02,67.03,46.95,73892.0,-15060.0,110237.0,-18.85,36.0,H-SC,2.42,154.0,-0.20,0.77,38.10,MH,ATH,POWER
9,BAJAJHFL,181.50,1.35,-19.57,84.58,48.45,136395.0,-39245.0,161261.0,-25.46,21.0,X-MC,8.79,64.0,-0.29,1.13,1.35,X40N,ATH,FINANCE
4,ALKYLAMINE,4546.37,0.67,-28.69,171.54,93.63,123500.0,-28968.0,71995.0,-23.56,28.0,H-SC,8.84,145.0,-0.23,0.51,7.23,SR,ATH,CHEMICALS
3,ACC,3906.00,0.66,-21.66,109.53,64.15,204186.0,-51531.0,186420.0,-54.13,52.0,X-SC,1.38,82.0,-0.25,1.31,4.57,XY24,BTT,CEMENT


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,EASEMYTRIP,26.4,-2.69,-54.14,217.69,45.70,176523.0,-95726.0,81089.0,-15.88,64.0,M-SC,23.37,205.0,-0.54,0.57,16.88,XY24,NTT,TRAVEL
44,JCHAC,2282.0,-0.83,-31.21,45.35,-0.01,39872.0,-39885.0,87920.0,17900.00,11.0,M-SC,0.56,233.0,-1.00,0.62,0.00,OX40N,NTT,AC
36,ICICIPRULI,790.0,-0.82,1.38,29.19,30.97,52300.0,2432.0,179170.0,-22.52,48.0,X-MC,2.18,75.0,0.05,1.26,14.11,X40,ATH,INSURANCE
84,WHIRLPOOL,2270.0,-0.77,-19.69,126.52,81.92,116643.0,-22605.0,92193.0,-52.97,23.0,M-SC,4.91,236.0,-0.19,0.65,9.14,XR,NTT,DURABLES
52,MEDANTA,1486.0,-0.76,1.47,22.27,24.08,28689.0,1872.0,128822.0,-6.77,41.0,X-SC,5.81,89.0,0.07,0.91,21.38,XY24,NTT,HEALTHCARE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.28,-0.33,107.40,106.72,132031.0,-402.0,122934.0,-50.63,38.0,H-SC,1.70,139.0,-0.00,0.86,22.98,AR,ATH,MISC
37,IEX,219.00,-0.11,0.69,47.60,48.63,95837.0,1384.0,201338.0,-32.92,65.0,H-SC,14.79,136.0,0.01,1.41,12.32,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,0.05,-0.54,36.36,35.63,75791.0,-1127.0,208447.0,-14.53,46.0,H-MC,3.61,119.0,-0.01,1.46,16.04,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.14,-5.79,120.70,107.91,167883.0,-8553.0,139091.0,-23.39,33.0,M-SC,11.13,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.25,-8.91,9.80,0.02,15582.0,-15554.0,159005.0,142.75,66.0,M-SC,14.25,234.0,-1.00,1.12,38.31,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.61,-16.05,19.22,0.09,16261.0,-16171.0,84606.0,96.37,42.0,M-SC,2.97,253.0,-0.99,0.59,11.15,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.02,-23.01,47.38,13.47,98375.0,-62037.0,207630.0,-68.31,32.0,H-SC,2.15,158.0,-0.63,1.46,5.57,XY24,NTT,PAINTS
67,SIS,528.00,-0.18,-25.42,62.59,21.26,51831.0,-28221.0,82811.0,1951.81,43.0,H-SC,3.97,166.0,-0.54,0.58,11.89,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,0.43,3.05,67.16,72.27,117274.0,5171.0,174619.0,-11.52,62.0,M-LC,3.15,99.0,0.04,1.23,9.42,XR,NTT,IT
37,IEX,219.0,-0.11,0.69,47.60,48.63,95837.0,1384.0,201338.0,-32.92,65.0,H-SC,14.79,136.0,0.01,1.41,12.32,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.43,3.05,67.16,72.27,117274.0,5171.0,174619.0,-11.52,62.0,M-LC,3.15,99.0,0.04,1.23,9.42,XR,NTT,IT
37,IEX,219.00,-0.11,0.69,47.60,48.63,95837.0,1384.0,201338.0,-32.92,65.0,H-SC,14.79,136.0,0.01,1.41,12.32,XR,NTT,MISC
38,INDIAMART,4810.62,-0.28,-0.33,107.40,106.72,132031.0,-402.0,122934.0,-50.63,38.0,H-SC,1.70,139.0,-0.00,0.86,22.98,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,0.05,-0.54,36.36,35.63,75791.0,-1127.0,208447.0,-14.53,46.0,H-MC,3.61,119.0,-0.01,1.46,16.04,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.14,-5.79,120.70,107.91,167883.0,-8553.0,139091.0,-23.39,33.0,M-SC,11.13,216.0,-0.05,0.98,0.00,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.18,-37.88,117.65,35.20,94036.0,-48741.0,79929.0,4.84,17.0,X-SC,14.26,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,1.35,-19.57,84.58,48.45,136395.0,-39245.0,161261.0,-25.46,21.0,X-MC,8.79,64.0,-0.29,1.13,1.35,X40N,ATH,FINANCE
8,AWL,485.00,0.22,-20.89,94.04,53.50,224475.0,-63049.0,238702.0,-61.72,30.0,X-MC,2.01,58.0,-0.28,1.68,6.02,XY24,NTT,FMCG
54,PAGEIND,51605.07,-0.23,-8.34,37.54,26.07,56340.0,-13660.0,150080.0,-32.53,30.0,X-MC,10.47,55.0,-0.24,1.05,0.31,X40,ATH,APPARELS
55,PGHH,17907.65,0.06,-5.09,40.93,33.76,78015.0,-10215.0,190605.0,-33.05,32.0,X-MC,4.49,57.0,-0.13,1.34,0.06,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.0,0.66,-21.66,109.53,64.15,204186.0,-51531.0,186420.0,-54.13,52.0,X-SC,1.38,82.0,-0.25,1.31,4.57,XY24,BTT,CEMENT
43,ITC,452.0,-0.04,-2.07,12.77,10.44,25028.0,-4147.0,195991.0,-41.56,40.0,X-LC,1.51,5.0,-0.17,1.38,3.38,X40,NTT,FMCG
15,CAMS,4762.0,-0.28,0.63,24.01,24.79,60851.0,1584.0,253440.0,-4.14,45.0,X-SC,1.89,86.0,0.03,1.78,23.91,X40N,NTT,MISC
8,AWL,485.0,0.22,-20.89,94.04,53.50,224475.0,-63049.0,238702.0,-61.72,30.0,X-MC,2.01,58.0,-0.28,1.68,6.02,XY24,NTT,FMCG
36,ICICIPRULI,790.0,-0.82,1.38,29.19,30.97,52300.0,2432.0,179170.0,-22.52,48.0,X-MC,2.18,75.0,0.05,1.26,14.11,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.18,-37.88,117.65,35.20,94036.0,-48741.0,79929.0,4.84,17.0,X-SC,14.26,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
56,QUESS,424.00,-0.42,-29.76,103.36,42.84,47196.0,-19344.0,45662.0,-54.27,38.0,X-SC,37.41,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
55,PGHH,17907.65,0.06,-5.09,40.93,33.76,78015.0,-10215.0,190605.0,-33.05,32.0,X-MC,4.49,57.0,-0.13,1.34,0.06,X40,ATH,FMCG
54,PAGEIND,51605.07,-0.23,-8.34,37.54,26.07,56340.0,-13660.0,150080.0,-32.53,30.0,X-MC,10.47,55.0,-0.24,1.05,0.31,X40,ATH,APPARELS
19,COLPAL,3726.84,-0.04,-18.08,72.75,41.51,156951.0,-47625.0,215740.0,-2.34,41.0,X-MC,7.20,61.0,-0.30,1.52,1.24,XY25,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.42,-29.76,103.36,42.84,47196.0,-19344.0,45662.0,-54.27,38.0,X-SC,37.41,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-0.04,-47.41,189.58,52.31,144739.0,-68813.0,76347.0,-44.05,35.0,X-SC,5.01,91.0,-0.48,0.54,2.19,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.18,-37.88,117.65,35.20,94036.0,-48741.0,79929.0,4.84,17.0,X-SC,14.26,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,-0.39,-7.28,31.12,21.58,40438.0,-10197.0,129942.0,-53.27,33.0,X-MC,5.79,56.0,-0.25,0.91,15.04,X40N,ATH,IT
52,MEDANTA,1486.00,-0.76,1.47,22.27,24.08,28689.0,1872.0,128822.0,-6.77,41.0,X-SC,5.81,89.0,0.07,0.91,21.38,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.25,-14.59,39.65,19.28,117165.0,-50458.0,295498.0,-25.57,59.0,X-LC,6.75,1.0,-0.43,2.08,9.24,X40,ATH,IT
41,INFY,2275.00,0.29,6.57,45.32,54.87,151120.0,20564.0,333452.0,-17.09,61.0,X-LC,2.69,2.0,0.14,2.34,15.40,X40,BTT,IT
76,TMPV,600.00,-0.66,-14.85,66.97,42.18,157630.0,-41036.0,235374.0,-23.08,33.0,X-LC,3.10,3.0,-0.26,1.65,1.96,XY24,NTT,AUTO
82,VBL,671.64,-0.26,-3.01,39.88,35.67,122167.0,-9506.0,306336.0,-14.52,60.0,X-LC,5.38,4.0,-0.08,2.15,10.43,X40N,ATH,FMCG
43,ITC,452.00,-0.04,-2.07,12.77,10.44,25028.0,-4147.0,195991.0,-41.56,40.0,X-LC,1.51,5.0,-0.17,1.38,3.38,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.12,-37.30,111.95,32.89,54192.0,-28800.0,48407.0,-697.80,60.0,L-MC,6.01,259.0,-0.53,0.34,33.31,XR,NTT,BANKS
6,ASIANTILES,137.00,0.34,-14.60,112.73,81.67,90313.0,-13696.0,80114.0,7200.00,46.0,L-SC,18.79,271.0,-0.15,0.56,55.14,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.18,-12.24,59.00,39.54,52379.0,-12381.0,88778.0,-31.30,63.0,M-SC,6.51,220.0,-0.24,0.62,25.19,AR,ATH,AUTO
51,MASFIN,398.61,0.22,-2.34,24.98,22.05,23902.0,-2295.0,95685.0,-15.99,59.0,H-SC,6.72,164.0,-0.10,0.67,39.09,XR,ATH,FINANCE
70,SURYODAY,216.00,0.00,-18.96,51.81,23.03,75190.0,-33945.0,145126.0,57.89,50.0,H-SC,10.00,167.0,-0.45,1.02,43.99,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,0.25,-8.91,9.80,0.02,15582.0,-15554.0,159005.0,142.75,66.0,M-SC,14.25,234.0,-1.00,1.12,38.31,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.02,3.25,17.14,20.95,44554.0,8175.0,259943.0,2.18,79.0,X-LC,17.34,31.0,0.18,1.83,40.04,X40,ATH,PAINTS
13,BSOFT,831.70,5.06,-26.58,105.38,50.79,108817.0,-37391.0,103262.0,-5.68,65.0,H-SC,6.39,151.0,-0.34,0.73,20.23,XR,ATH,IT
50,LTIM,7201.88,0.44,13.18,16.33,31.66,45495.0,32444.0,278595.0,8.90,70.0,H-LC,14.96,43.0,0.71,1.96,54.58,X200,ATH,IT
0,5PAISA,593.00,0.37,-35.84,75.96,12.90,106490.0,-78304.0,140192.0,106.73,55.0,H-SC,11.92,161.0,-0.74,0.98,16.57,OX40N,NTT,FINANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.00
1,25,44.79
2,50,75.66


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.44
MC    28.97
LC    25.61
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.70
X40      21.26
X40N     11.91
XR        9.67
AR        9.10
XY25      9.09
OX40N     7.58
X200      1.96
SR        0.98
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.92
X-MC    22.48
X-LC    20.26
M-SC    11.95
X-SC     8.18
H-MC     4.77
H-LC     3.14
L-SC     1.39
M-MC     1.38
M-LC     1.23
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.74,-5.65,39.81
IT,13.98,-13.75,72.47
FINANCE,9.39,-15.86,64.81
MISC,7.42,-22.95,73.32
ELECTRICAL,6.02,-9.63,49.78
PAINTS,6.00,-8.33,24.42
INSURANCE,4.39,-1.37,37.10
PHARMA,3.90,-0.45,32.60
AUTO,3.37,-19.64,70.94


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3096602.0,21
XR,1285689.0,13
AR,1277377.0,10
X40,1013763.0,14
X40N,751907.0,9
OX40N,712142.0,10
XY25,347567.0,6
SR,280882.0,2
MH,73892.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3572132.0,25
M-SC,1359295.0,15
X-MC,1282454.0,16
X-LC,879999.0,11
X-SC,756345.0,8
H-MC,400030.0,3
L-SC,263416.0,3
M-LC,117274.0,1
H-LC,111404.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1235892.0      6
           AR         882140.0      5
M-SC       XY24       774300.0      6
H-SC       XR         771343.0      7
X-MC       X40        453011.0      7
           XY24       385302.0      3
X-LC       X40        382523.0      5
H-SC       OX40N      327983.0      4
X-SC       X40N       298045.0      3
M-SC       OX40N      297531.0      5
H-SC       SR         280882.0      2
X-SC       XY24       280071.0      3
X-MC       X40N       260179.0      4
X-LC       XY24       228963.0      2
H-MC       AR         207956.0      2
X-LC       X40N       193683.0      2
H-MC       XY24       192074.0      1
X-MC       XY25       183962.0      2
X-SC       X40        178229.0      2
L-SC       XR         176788.0      2
M-SC       XR         166092.0      2
           AR         121372.0      2
M-LC       XR         117274.0      1
L-SC       OX40N       86628.0      1
X-LC       XY25        74830.0      2
H-SC       MH          73892.0      1
H-LC       AR          65909.0      1
L-MC       XR          54192.0      1
M-MC       XY25        48697.0      1
H-LC       X200        45495.0      1
L-LC       XY25        40078.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 33.0 seconds
